In [1]:
# !wget https://huggingface.co/datasets/syafie-nzm/crawl-jurnaldbp/resolve/main/jurnaldbp.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/mjpharm.org.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/myjgeosc.com.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/myjsustainagri.com.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/akademisains.gov.my.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/crossref-pdf.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/Kamus_Dewan_Bahasa_Edisi_Keempat_pdf.pdf
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/melayu-pdf.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/majcafe.com.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/myjms.mohe.gov.my.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/newera.edu.my.jsonl

In [2]:
from glob import glob
from tqdm import tqdm
import json
import os
import numpy as np
from transformers import AutoTokenizer
from streaming import MDSWriter
from tqdm import tqdm
import msgspec
import json
import re

http_errors = [
        "400 Bad Request", "401 Unauthorized", "402 Payment Required", "403 Forbidden", "404 Not Found",
        "405 Method Not Allowed", "406 Not Acceptable", "407 Proxy Authentication Required", "408 Request Timeout",
        "409 Conflict", "410 Gone", "411 Length Required", "412 Precondition Failed", "413 Payload Too Large",
        "414 URI Too Long", "415 Unsupported Media Type", "416 Range Not Satisfiable", "417 Expectation Failed",
        "418 I'm a teapot", "421 Misdirected Request", "422 Unprocessable Entity", "423 Locked", "424 Failed Dependency",
        "425 Too Early", "426 Upgrade Required", "428 Precondition Required", "429 Too Many Requests",
        "431 Request Header Fields Too Large", "451 Unavailable For Legal Reasons", "500 Internal Server Error",
        "501 Not Implemented", "502 Bad Gateway", "503 Service Unavailable", "504 Gateway Timeout",
        "505 HTTP Version Not Supported", "506 Variant Also Negotiates", "507 Insufficient Storage",
        "508 Loop Detected", "510 Not Extended", "511 Network Authentication Required"
    ]

rejected = [
    'Internal Server Error',
    '__NOEDITSECTION__',
    'enter your username and password',
    'forgotten your password',
    'cookies enabled',
    'enable JavaScript in your browser.',
    'The page cannot be displayed',
    'site or edit the error_page',
    'Request unsuccessful',
]

rejected.extend(http_errors)

def replace_multiple(input_string, pattern =r"\s{6,}", replace = '   '):
    return re.sub(pattern, replace, input_string)

def replace(string):
    string = replace_multiple(string.replace('…', '.'))
    string = replace_multiple(string, pattern = r"\.{6,}", replace = '...')
    return string

def reject(string):
    if any([r in string for r in rejected]):
        return True
    return False

In [3]:
from streaming.base.format.mds.encodings import Encoding, _encodings

class UInt32(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint32)

_encodings['uint32'] = UInt32

In [4]:
files = [
    'jurnaldbp.jsonl',
    'mjpharm.org.jsonl',
    'myjgeosc.com.jsonl',
    'myjsustainagri.com.jsonl',
    'akademisains.gov.my.jsonl',
    'crossref-pdf.jsonl',
    'Kamus_Dewan_Bahasa_Edisi_Keempat_pdf.pdf',
    'melayu-pdf.jsonl',
    'majcafe.com.jsonl',
    'myjms.mohe.gov.my.jsonl',
    'newera.edu.my.jsonl'
]

In [5]:
columns = {
    'input_ids': 'uint32',
}
compression = 'zstd'
hashes = 'sha1', 'xxh64'

In [6]:
def partition(text, size = 500):
    splitted = text.split()
    return [' '.join(splitted[i: i + size]) for i in range(0, len(splitted), size)]

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen1.5-0.5B')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# with open('extra.jsonl', 'w') as fopen_l:
#     for f in files:
#         with open(f, encoding = "ISO-8859-1") as fopen:
#             for l in tqdm(fopen):
#                 try:
#                     l = msgspec.json.decode(l)
#                     if reject(l):
#                         continue

#                     l = replace(l.strip())
#                     if len(l) < 3:
#                         continue
                        
#                     data = '<|endoftext|>' + l + '<|endoftext|>'
#                     partitioned = partition(data)
#                     for p in partitioned:
#                         data = {
#                             'text': p,
#                         }
#                         fopen_l.write(f'{json.dumps(data)}\n')
#                         fopen_l.flush()
#                 except Exception as e:
#                     print(e)
#                     pass

In [ ]:
split_by = 100000

In [ ]:
!rm -rf partitions-extra
!mkdir partitions-extra

In [ ]:
index = 0
count = 0
a = open(f'partitions-extra/combined-lm-{index}.jsonl', 'w')

with open('extra.jsonl') as fopen:
    for l in tqdm(fopen):
        a.write(l)
        a.flush()
        count += 1
        if count >= split_by:
            a.close()
            index += 1
            count = 0
            a = open(f'partitions-extra/combined-lm-{index}.jsonl', 'w')
            
a.close()

In [14]:
block_size = 16384

def read_dataset(train_file, block_size = block_size):
    
    tokenizer = AutoTokenizer.from_pretrained(
        'Qwen/Qwen1.5-0.5B',
    )
    tokenizer.add_bos_token = False
    tokenizer.add_eos_token = False
    text_column_name = 'text'
    temp = []
    with open(train_file) as fopen:
        for l in fopen:
            l = msgspec.json.decode(l)
            tokenized = tokenizer(l['text'])['input_ids']
            temp.extend(tokenized)
            while len(temp) >= block_size:
                block = temp[:block_size]
                temp = temp[block_size:]
                if len(block) == block_size:
                    yield np.array(block).astype(np.uint32)

In [15]:
files = sorted(glob('partitions-extra/combined-lm-*.jsonl'), key = lambda x: int(x.split('-')[-1].replace('.jsonl', '')))
files

['partitions-extra/combined-lm-0.jsonl',
 'partitions-extra/combined-lm-1.jsonl',
 'partitions-extra/combined-lm-2.jsonl',
 'partitions-extra/combined-lm-3.jsonl',
 'partitions-extra/combined-lm-4.jsonl',
 'partitions-extra/combined-lm-5.jsonl',
 'partitions-extra/combined-lm-6.jsonl',
 'partitions-extra/combined-lm-7.jsonl',
 'partitions-extra/combined-lm-8.jsonl',
 'partitions-extra/combined-lm-9.jsonl',
 'partitions-extra/combined-lm-10.jsonl',
 'partitions-extra/combined-lm-11.jsonl',
 'partitions-extra/combined-lm-12.jsonl']

In [16]:
r = next(read_dataset(files[0]))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
tokenizer.decode(r)

'<|endoftext|>288 KANUN [DISEMBER 2014][ULASAN BUKU]ULASAN BUKU Mohd Bakri IshakUndang-undang dan Pengurusan Sisa Industri di Malaysia dan Eropah, 2013.180 halaman, ISBN9789834610135Pengulas: Shamsuddin Suhor sudin@ukm.edu.myFakulti Undang-undang,Universiti Kebangsaan Malaysia,43650 Bangi, Selangor Darul Ehsan, MALAYSIA.Buku ini mengandungi 12 bab yang memperkatakan pelbagai perkara berkaitan dengan sisa industri. Setiap bab dilihat mampu berdiri dengan sendiri bagi menceritakan sesuatu yang berkaitan dengan sisa, sama ada sisa dari sudut falsafah seperti yang dijelaskan dalam Bab 2, atau sisa industri dari sudut perundangan yang melibatkan Bab 3, 4, 5 dan 6, atau pengurusan sisa industri melalui pendekatan bukan perundangan seperti dalam Bab 9, 10, dan 11. Seterusnya buku ini membincangkan kes yang menyentuh isu pencemaran alam sekitar secara umum dan/atau yang khusus mengenai sisa industri yang telah diputuskan oleh mahkamah di negara ini seperti yang dibincangkan dalam Bab 7 dan 8 b

In [18]:
r

array([151643,     17,     23, ...,  88994,  60593,     12], dtype=uint32)

In [19]:
!rm -rf tokenized_extra
!mkdir tokenized_extra

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
def loop(files):
    files, index = files
    out_root = f'tokenized_extra/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    with MDSWriter(out=out_root, columns=columns, compression=compression, hashes=hashes) as out:
        for f in files:
            for block in tqdm(read_dataset(train_file = f)):
                sample = {
                    'input_ids': block
                }
                out.write(sample)

In [21]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [22]:
import mp
mp.multiprocessing(files, loop, cores = min(len(files), 20), returned = False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [23]:
from streaming import StreamingDataset

dataset = StreamingDataset(local = 'tokenized_extra/tokenized-0')

/home/ubuntu/.local/lib/python3.10/site-packages/streaming/base/dataset.py:397: UserWarning: Because `predownload` was not specified, it will default to 8*batch_size if batch_size is not None, otherwise 64. Prior to Streaming v0.7.0, `predownload` defaulted to max(batch_size, 256 * batch_size // num_canonical_nodes).
  warnings.warn(f'Because `predownload` was not specified, it will default to ' +


In [24]:
dataset[0]

{'input_ids': array([151643,     17,     23, ...,  88994,  60593,     12], dtype=uint32)}